# Workshop Environment Setup

This notebook is part of a workshop demonstrating pgvector with Amazon Aurora PostgreSQL and Amazon Bedrock. We'll automatically configure your environment using AWS Secrets Manager and environment variables.

In [ ]:
%%writefile requirements.txt
psycopg[binary]==3.1.16
pandas>=2.0.0
numpy>=1.24.0
boto3>=1.34.0
tqdm>=4.66.0

In [ ]:
import sys
import subprocess
import json
import os

def install_packages():
    """Install required Python packages"""
    print("Installing required packages...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
        print("\n✅ Package installation complete")
    except subprocess.CalledProcessError as e:
        print(f"Error installing packages: {e}")
        raise

# Install packages if needed
try:
    import psycopg
    print("psycopg3 is already installed")
except ImportError:
    print("Installing required packages including psycopg3...")
    install_packages()

# Import required packages
import boto3
import botocore

def setup_workshop_environment():
    """Set up workshop environment variables using AWS Secrets Manager"""
    try:
        # Get AWS region from instance metadata
        region = boto3.session.Session().region_name
        if not region:
            print("⚠️ Could not determine AWS region. Using default region us-west-2")
            region = 'us-west-2'
        
        print(f"Using AWS region: {region}")
        
        # Initialize AWS clients
        secrets_client = boto3.client('secretsmanager', region_name=region)
        rds_client = boto3.client('rds', region_name=region)
        
        # Get database credentials from Secrets Manager
        secret_name = "apg-pgvector-secret-RIV"
        try:
            secret_response = secrets_client.get_secret_value(SecretId=secret_name)
            db_credentials = json.loads(secret_response['SecretString'])
            
            # Set environment variables
            os.environ['PGUSER'] = db_credentials.get('username')
            os.environ['PGPASSWORD'] = db_credentials.get('password')
            os.environ['PGDATABASE'] = 'postgres'
            
            print("✅ Database credentials retrieved from Secrets Manager")
            
        except botocore.exceptions.ClientError as e:
            print(f"⚠️ Error accessing Secrets Manager: {e}")
            return False
        
        # Get cluster endpoint
        cluster_id = "apg-pgvector-riv"
        try:
            response = rds_client.describe_db_clusters(
                DBClusterIdentifier=cluster_id
            )
            
            if response['DBClusters']:
                cluster = response['DBClusters'][0]
                os.environ['PGHOST'] = cluster['Endpoint']
                os.environ['PGPORT'] = str(cluster['Port'])
                print("✅ Database endpoint information retrieved")
            
        except botocore.exceptions.ClientError as e:
            print(f"⚠️ Error accessing RDS: {e}")
            return False
        
        # Verify environment variables
        required_vars = ['PGHOST', 'PGPORT', 'PGUSER', 'PGPASSWORD', 'PGDATABASE']
        missing_vars = [var for var in required_vars if not os.getenv(var)]
        
        if missing_vars:
            print("\n⚠️ Missing required environment variables:")
            print("\n".join(missing_vars))
            return False
        
        print("\n✅ Workshop environment successfully configured")
        print(f"\nDatabase Host: {os.getenv('PGHOST')}")
        print(f"Database Port: {os.getenv('PGPORT')}")
        print(f"Database Name: {os.getenv('PGDATABASE')}")
        print(f"Database User: {os.getenv('PGUSER')}")
        return True
        
    except Exception as e:
        print(f"\n⚠️ Error setting up workshop environment: {str(e)}")
        print("\nIf you're having issues, please raise your hand for workshop support.")
        return False

# Set up the workshop environment
if setup_workshop_environment():
    print("\n🚀 You're ready to proceed with the workshop!")
else:
    print("\n⚠️ Environment setup failed. Please seek assistance from the workshop staff.")

## Verify Package Installation

Let's verify that all required packages are installed correctly.

In [ ]:
# Verify all required packages
try:
    import psycopg
    import pandas as pd
    import numpy as np
    import boto3
    import tqdm

    print("Installed package versions:")
    print(f"psycopg: {psycopg.__version__}")
    print(f"pandas: {pd.__version__}")
    print(f"numpy: {np.__version__}")
    print(f"boto3: {boto3.__version__}")
    print(f"tqdm: {tqdm.__version__}")
    print("\n✅ All required packages are installed correctly")
except ImportError as e:
    print(f"⚠️ Error importing packages: {e}")
    print("Please seek assistance from the workshop staff")

# Product Catalog Setup and Embedding Generation with pgvector and Amazon Bedrock

This notebook demonstrates how to:
1. Set up pgvector extension in Aurora PostgreSQL
2. Create and configure a product catalog table with vector storage capabilities
3. Load product data from a compressed CSV file
4. Generate embeddings using Amazon Bedrock's Titan Text Embeddings model
5. Store and index the embeddings for vector similarity search

## Prerequisites
- Aurora PostgreSQL cluster with pgvector extension available
- AWS credentials configured with access to Amazon Bedrock
- Product catalog data in CSV format (compressed with gzip)
- Required Python packages: psycopg[binary], boto3, pandas, numpy

## Import Required Libraries and Configure Environment

In [ ]:
import os
import boto3
import json
import pandas as pd
import numpy as np
import psycopg
from psycopg.rows import dict_row
from psycopg.adapt import Dumper, PyFormat
import gzip
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Configure numpy array adaptation for PostgreSQL
class NumpyArrayDumper(Dumper):
    def dump(self, obj):
        return str(obj.tolist())

psycopg.adapters.register_dumper(np.ndarray, NumpyArrayDumper)

# Database connection parameters (from environment variables)
DB_PARAMS = {
    'dbname': os.getenv('PGDATABASE', 'postgres'),
    'user': os.getenv('PGUSER'),
    'password': os.getenv('PGPASSWORD'),
    'host': os.getenv('PGHOST'),
    'port': os.getenv('PGPORT', '5432')
}

# Initialize Bedrock client
bedrock = boto3.client('bedrock-runtime')

## PostgreSQL Setup Functions

First, we'll define functions to handle our PostgreSQL operations using psycopg3. Key differences from psycopg2 include:
1. Context manager support for connections and cursors
2. Native support for async/await (though we'll use sync here)
3. Improved type adaptation system
4. Better error handling and diagnostics

In [ ]:
def get_db_connection():
    """Create and return a PostgreSQL database connection using psycopg3"""
    try:
        conn = psycopg.connect(
            **DB_PARAMS,
            autocommit=True,
            row_factory=dict_row  # Return rows as dictionaries
        )
        return conn
    except Exception as e:
        print(f"Error connecting to database: {str(e)}")
        raise

def setup_database():
    """Set up the database with required extension and schema"""
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            # Create vector extension if it doesn't exist
            cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
            print("✅ Vector extension created/verified")
            
            # Create schema if it doesn't exist
            cur.execute("CREATE SCHEMA IF NOT EXISTS bedrock_integration;")
            print("✅ Schema created/verified")
            
            # Create product catalog table with vector support
            cur.execute("""
                CREATE TABLE IF NOT EXISTS bedrock_integration.product_catalog (
                    "productId" VARCHAR(255),
                    product_description TEXT,
                    imgUrl TEXT,
                    productURL TEXT,
                    stars NUMERIC,
                    reviews INT,
                    price NUMERIC,
                    category_id INT,
                    isBestSeller BOOLEAN,
                    boughtInLastMonth INT,
                    category_name VARCHAR(255),
                    quantity INT,
                    embedding vector(1024)
                );
            """)
            print("✅ Product catalog table created/verified")
            
            # Create HNSW index for vector similarity search
            cur.execute("""
                CREATE INDEX IF NOT EXISTS product_catalog_embedding_idx 
                ON bedrock_integration.product_catalog 
                USING hnsw (embedding vector_cosine_ops);
            """)
            print("✅ HNSW index created/verified")

# Execute setup
setup_database()

## Load Product Catalog Data

Using psycopg3's improved batch execution capabilities for data loading:

In [ ]:
def load_product_data(file_path='datasets/product_catalog.csv.gz', chunk_size=1000):
    """Load product catalog data from gzipped CSV file into PostgreSQL"""
    
    # First, check if we already have data
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT COUNT(*) FROM bedrock_integration.product_catalog;")
            count = cur.fetchone()['count']
            if count > 0:
                print(f"Table already contains {count} rows. Skipping data load.")
                return

    # Read the gzipped CSV file in chunks
    chunks = pd.read_csv(
        file_path, 
        compression='gzip',
        chunksize=chunk_size
    )
    
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            total_rows = 0
            for chunk in tqdm(chunks, desc="Loading data chunks"):
                # Prepare values for insertion
                values = [
                    tuple(row) for row in chunk.replace({np.nan: None}).values
                ]
                
                # Using psycopg3's execute_batch for better performance
                psycopg.execute_batch(cur, """
                    INSERT INTO bedrock_integration.product_catalog (
                        "productId", product_description, imgUrl, productURL,
                        stars, reviews, price, category_id, isBestSeller,
                        boughtInLastMonth, category_name, quantity
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                "", values, page_size=100)
                
                total_rows += len(values)
            
            print(f"✅ Successfully loaded {total_rows} products")

# Execute data load
load_product_data()

## Generate Embeddings with Amazon Bedrock

Using psycopg3's improved connection handling for embedding generation and storage:

In [ ]:
def get_embedding(text):
    """Generate embedding for a single text using Amazon Bedrock"""
    try:
        response = bedrock.invoke_model(
            modelId="amazon.titan-embed-text-v2:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({"inputText": text})
        )
        response_body = json.loads(response['body'].read())
        return np.array(response_body['embedding'])
    except Exception as e:
        print(f"Error generating embedding for text: {text[:50]}...")
        print(f"Error message: {str(e)}")
        return None

def process_products_batch(products, max_workers=10):
    """Process a batch of products to generate and store embeddings"""
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        embeddings = list(tqdm(
            executor.map(get_embedding, products['product_description']),
            total=len(products),
            desc="Generating embeddings"
        ))
    
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            # Using psycopg3's execute_batch for better performance
            updates = [
                (embedding, product_id)
                for embedding, product_id in zip(embeddings, products['productId'])
                if embedding is not None
            ]
            
            psycopg.execute_batch(cur, """
                UPDATE bedrock_integration.product_catalog
                SET embedding = %s
                WHERE "productId" = %s;
            "", updates, page_size=100)
    
    return len([e for e in embeddings if e is not None])

def generate_all_embeddings(batch_size=100):
    """Generate embeddings for all products without existing embeddings"""
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            # Get products without embeddings
            cur.execute("""
                SELECT "productId", product_description
                FROM bedrock_integration.product_catalog
                WHERE embedding IS NULL;
            """)
            products = pd.DataFrame(cur.fetchall())
            
            if len(products) == 0:
                print("No products need embeddings generated")
                return
            
            print(f"Generating embeddings for {len(products)} products")
            
            # Process in batches
            total_processed = 0
            for i in range(0, len(products), batch_size):
                batch = products.iloc[i:i+batch_size]
                processed = process_products_batch(batch)
                total_processed += processed
                print(f"Batch {i//batch_size + 1}: Processed {processed} products")
            
            print(f"✅ Successfully generated embeddings for {total_processed} products")

# Generate embeddings for all products
generate_all_embeddings()

## Verify Setup

Using psycopg3's improved row factory for better data access:

In [ ]:
def verify_setup():
    """Verify the database setup and embedding generation"""
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            # Check total products
            cur.execute("""
                SELECT COUNT(*)
                FROM bedrock_integration.product_catalog;
            """)
            total_products = cur.fetchone()['count']
            print(f"Total products in catalog: {total_products}")
            
            # Check products with embeddings
            cur.execute("""
                SELECT COUNT(*)
                FROM bedrock_integration.product_catalog
                WHERE embedding IS NOT NULL;
            """)
            products_with_embeddings = cur.fetchone()['count']
            print(f"Products with embeddings: {products_with_embeddings}")
            
            # Check embedding dimensionality
            if products_with_embeddings > 0:
                cur.execute("""
                    SELECT array_length(embedding, 1)
                    FROM bedrock_integration.product_catalog
                    WHERE embedding IS NOT NULL
                    LIMIT 1;
                """)
                embedding_dim = cur.fetchone()['array_length']
                print(f"Embedding dimensionality: {embedding_dim}")
            
            # Check HNSW index
            cur.execute("""
                SELECT indexname, indexdef
                FROM pg_indexes
                WHERE tablename = 'product_catalog'
                AND indexname = 'product_catalog_embedding_idx';
            """)
            index_info = cur.fetchone()
            if index_info:
                print("✅ HNSW index is properly configured")
            else:
                print("⚠️ HNSW index is missing")
            
            # Calculate completion percentage
            if total_products > 0:
                completion_pct = (products_with_embeddings / total_products) * 100
                print(f"\nEmbedding generation progress: {completion_pct:.1f}%")
            
            # Sample query to verify vector similarity search
            if products_with_embeddings > 0:
                print("\nTesting vector similarity search...")
                cur.execute("""
                    SELECT "productId", product_description, stars
                    FROM bedrock_integration.product_catalog
                    WHERE embedding IS NOT NULL
                    ORDER BY RANDOM()
                    LIMIT 1;
                """)
                sample_product = cur.fetchone()
                
                if sample_product:
                    print(f"\nSample product: {sample_product['product_description'][:100]}...")
                    
                    # Get embedding for sample product
                    cur.execute("""
                        SELECT embedding
                        FROM bedrock_integration.product_catalog
                        WHERE "productId" = %s;
                    "", (sample_product['productId'],))
                    sample_embedding = cur.fetchone()['embedding']
                    
                    # Find similar products using pgvector's cosine similarity operator
                    cur.execute("""
                        SELECT "productId", product_description,
                               1 - (embedding <=> %s) as similarity
                        FROM bedrock_integration.product_catalog
                        WHERE "productId" != %s
                        ORDER BY embedding <=> %s
                        LIMIT 3;
                    "", (sample_embedding, sample_product['productId'], sample_embedding))
                    
                    similar_products = cur.fetchall()
                    print("\nSimilar products:")
                    for prod in similar_products:
                        print(f"Similarity: {prod['similarity']:.3f}")
                        print(f"Description: {prod['product_description'][:100]}...\n")

# Run verification
verify_setup()

## Add Sample Semantic Search Function

Let's add a utility function to perform semantic search using natural language queries:

In [ ]:
def semantic_search(query_text, limit=5):
    """Search for products using semantic similarity to the query text"""
    # Generate embedding for the query text
    query_embedding = get_embedding(query_text)
    if query_embedding is None:
        print("Failed to generate embedding for query")
        return
    
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            # Search using vector similarity
            cur.execute("""
                SELECT 
                    "productId",
                    product_description,
                    stars,
                    price,
                    category_name,
                    1 - (embedding <=> %s) as similarity
                FROM bedrock_integration.product_catalog
                WHERE embedding IS NOT NULL
                ORDER BY embedding <=> %s
                LIMIT %s;
            "", (query_embedding, query_embedding, limit))
            
            results = cur.fetchall()
            
            if not results:
                print("No matching products found")
                return
            
            print(f"Top {len(results)} matches for: '{query_text}'\n")
            for result in results:
                print(f"Similarity: {result['similarity']:.3f}")
                print(f"Category: {result['category_name']}")
                print(f"Price: ${result['price']:.2f}")
                print(f"Stars: {result['stars']}")
                print(f"Description: {result['product_description'][:200]}...\n")

# Example usage
print("Example semantic search queries:")
queries = [
    "comfortable running shoes for marathon training",
    "stylish waterproof backpack for college",
    "professional chef knife set with wooden block"
]

for query in queries:
    print("\n" + "=" * 80)
    semantic_search(query, limit=3)

## Conclusion

In this notebook, we've successfully:
1. Set up pgvector extension in our Aurora PostgreSQL database
2. Created a product catalog table with vector storage capabilities
3. Loaded product data from a compressed CSV file
4. Generated embeddings using Amazon Bedrock's Titan Text Embeddings model
5. Stored and indexed the embeddings for efficient vector similarity search
6. Implemented semantic search functionality using natural language queries

The system demonstrates several advanced features:
- Efficient data loading with chunked processing
- Parallel embedding generation with thread pooling
- HNSW indexing for fast approximate nearest neighbor search
- Vector similarity search using pgvector's operators
- Natural language semantic search capabilities

For production use, consider:
- Adding error handling and retry logic for embedding generation
- Implementing batch processing for large datasets
- Monitoring embedding generation performance
- Setting up regular data updates and embedding refresh processes
- Implementing caching for frequently accessed embeddings
- Adding input validation and sanitization for search queries

The combination of pgvector and Amazon Bedrock provides a powerful foundation for building semantic search and recommendation systems that understand natural language queries and find relevant products based on semantic similarity rather than just keyword matching.